In [82]:
import sys
!conda install -q -y -c conda-forge --prefix {sys.prefix} ipyleaflet

# If you have JupyterLab, you will also need to install the JupyterLab extension:
# jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-leaflet

Solving environment: ...working... done

# All requested packages already installed.



In [83]:
import ipyleaflet
from birdy import WPSClient

from pprint import pprint

In [135]:
# pavics.ouranos.ca
# pavics_url = "https://pavics.ouranos.ca"
# finch_url = f'{pavics_url}/twitcher/ows/proxy/finch/wps'

# colibri
# Important note: this is a development server, uptime is not guaranteed
pavics_url = "http://colibri.crim.ca"
finch_url = f"{pavics_url}:8095/wps"

# Setting up the map

In [78]:
canada_center = (52.4292, -93.2959)
leaflet_map = ipyleaflet.Map(
    center=canada_center,
    basemap=ipyleaflet.basemaps.Stamen.Terrain,
    zoom=4,
)

leaflet_map

Map(basemap={'url': 'https://stamen-tiles-a.a.ssl.fastly.net/terrain/{z}/{x}/{y}.png', 'attribution': 'Map til…

### Note about ipyleaflet
The ipyleaflet object attributes are synced with the widgets. Try moving the map above and running the cell below to see the updated coordinates.

See the documentation at: https://ipyleaflet.readthedocs.io

In [15]:
leaflet_map.center

[54.00776876193478, -90.6591796875]

## Get a rectangle from the user

In [170]:
import ipyleaflet
from IPython.display import display
import ipywidgets as widgets

def get_rectangle():
    canada_center = (52.4292, -93.2959)
    m = ipyleaflet.Map(
        center=canada_center,
        basemap=ipyleaflet.basemaps.Stamen.Terrain,
        zoom=4,
    )
    
    # Create a new draw control
    draw_control = ipyleaflet.DrawControl()

    # disable some drawing inputs
    draw_control.polyline = {}
    draw_control.circlemarker = {}
    draw_control.polygon = {}

    draw_control.rectangle = {
        "shapeOptions": {
            "fillColor": "#4ae",
            "color": "#4ae",
            "fillOpacity": 0.3,
        }
    }

    output = widgets.Output(layout={'border': '1px solid black'})
    
    rectangle = {}

    # set drawing callback
    def callback(control, action, geo_json):
        if action == "created":
            # note: we can't close the map or remove it from the output
            # from this callback. The map keeps the focus, and the 
            # jupyter keyboard input is messed up.
            # So we set it very thin to make it disappear :)
            m.layout = {"max_height": "0"}
            with output:
                print("*User selected 1 rectangle*")
                rectangle.update(geo_json)

    draw_control.on_draw(callback)    

    m.add_control(draw_control)
    
    with output:
        print("Select a rectangle:")
        display(m)
        
    display(output)
    
    return rectangle

In [171]:
rectangle = get_rectangle()

Output(layout=Layout(border='1px solid black'))

In [172]:
import geopandas as gpd

rect = gpd.GeoDataFrame.from_features([rectangle])
bounds = rect.bounds
bounds

,minx,miny,maxx,maxy
0,-100.223837,50.373496,-87.477941,54.901882


## Call wps processes

In [173]:
finch = WPSClient(finch_url, progress=True)

In [174]:
help(finch.subset_bbox)

Help on method subset_bbox in module birdy.client.base:

subset_bbox(resource=None, lon0=0.0, lon1=360.0, lat0=-90.0, lat1=90.0, y0=None, y1=None, variable=None) method of birdy.client.base.WPSClient instance
    Return the data for which grid cells intersect the bounding box for each input dataset as well as the time range selected.
    
    Parameters
    ----------
    resource : ComplexData:mimetype:`application/x-netcdf`, :mimetype:`application/x-ogc-dods`
        NetCDF files, can be OPEnDAP urls.
    lon0 : float
        Minimum longitude.
    lon1 : float
        Maximum longitude.
    lat0 : float
        Minimum latitude.
    lat1 : float
        Maximum latitude.
    y0 : integer
        Initial year for temporal subsetting. Defaults to first year in file.
    y1 : integer
        Final year for temporal subsetting. Defaults to last year in file.
    variable : string
        Name of the variable in the NetCDF file.If not provided, all variables will be subsetted.
    
    R

### Subset both tasmin and tasmax datasets

In [175]:
tasmin = "http://colibri.crim.ca:8083/thredds/dodsC/birdhouse/nrcan/nrcan_canada_daily/nrcan_canada_daily_tasmin_1958.nc"
tasmax = "http://colibri.crim.ca:8083/thredds/dodsC/birdhouse/nrcan/nrcan_canada_daily/nrcan_canada_daily_tasmax_1958.nc"
lon0 = float(bounds.minx)
lon1 = float(bounds.maxx)
lat0 = float(bounds.miny)
lat1 = float(bounds.maxy)


In [176]:
result_tasmin = finch.subset_bbox(resource=tasmin, lon0=lon0, lon1=lon1, lat0=lat0, lat1=lat1)

In [177]:
# wait for process to complete before executing
tasmin_subset = result_tasmin.get().output
tasmin_subset

'https://colibri.crim.ca/wpsoutputs/d04a9b5e-d656-11e9-bed5-0242ac130004/nrcan_canada_daily_tasmin_1958_sub.nc'

In [178]:
result_tasmax = finch.subset_bbox(resource=tasmax, lon0=lon0, lon1=lon1, lat0=lat0, lat1=lat1)

In [180]:
# wait for process to complete before executing
tasmax_subset = result_tasmax.get().output
tasmax_subset

'https://colibri.crim.ca/wpsoutputs/e0da0306-d656-11e9-bed5-0242ac130004/nrcan_canada_daily_tasmax_1958_sub.nc'

### Launch a heat wave frequency analysis with the subsetted datasets

In [181]:
help(finch.heat_wave_frequency)

Help on method heat_wave_frequency in module birdy.client.base:

heat_wave_frequency(tasmin=None, tasmax=None, thresh_tasmin='22.0 degC', thresh_tasmax='30 degC', window=3, freq='YS') method of birdy.client.base.WPSClient instance
    Number of heat waves over a given period. A heat wave is defined as an event
        where the minimum and maximum daily temperature both exceeds specific thresholds
        over a minimum number of days.
    
    Parameters
    ----------
    tasmin : ComplexData:mimetype:`application/x-netcdf`
        NetCDF Files or archive (tar/zip) containing netCDF files.
    tasmax : ComplexData:mimetype:`application/x-netcdf`
        NetCDF Files or archive (tar/zip) containing netCDF files.
    thresh_tasmin : string
        threshold
    thresh_tasmax : string
        threshold
    window : integer
        window
    freq : {'YS', 'MS', 'QS-DEC', 'AS-JUL'}string
        Resampling frequency
    
    Returns
    -------
    output_netcdf : ComplexData:mimetype:`a

In [205]:
result = finch.heat_wave_frequency(tasmin=tasmin_subset, tasmax=tasmax_subset)

In [198]:
ds, logs = result.get(asobj=True)

In [199]:
ds

<xarray.Dataset>
Dimensions:              (lat: 55, lon: 153, time: 1)
Coordinates:
  * time                 (time) datetime64[ns] 1958-01-01
  * lon                  (lon) float32 -100.208336 -100.125 ... -87.54167
  * lat                  (lat) float32 54.875 54.791668 ... 50.458332 50.375
Data variables:
    heat_wave_frequency  (time, lat, lon) float64 ...
Attributes:
    Conventions:                     CF-1.5
    title:                           NRCAN 10km Gridded Climate Dataset
    history:                         2012-10-22T09:11:24: Convert from origin...
    institution:                     NRCAN
    source:                          ANUSPLIN
    redistribution:                  Redistribution policy unknown. For inter...
    DODS_EXTRA.Unlimited_Dimension:  time

### Showing wms layers on a map

If the results were made available to thredds, we could display them as a wms service without downloading the data.

For this example, we just pick a random dataset from thredds and display it on a map.

In [206]:
def show_wms(url):
    canada_center = (52.4292, -93.2959)
    
    wms = ipyleaflet.WMSLayer(
        url=url,
        layers="tasmax",
        styles="boxfill/rainbow",
        format="image/png",
        transparent=True,
        version="1.3.0",
    )
    
    m = ipyleaflet.Map(
        layers=(wms, ),
        center=canada_center,
        basemap=ipyleaflet.basemaps.Stamen.Terrain,
        zoom=4,
    )
    return m
    
m = show_wms(f"{pavics_url}/thredds/wms/birdhouse/nrcan/nrcan_canada_daily/nrcan_canada_daily_tasmax_1958.nc")
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …